In [73]:
# Loading packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.ndimage.interpolation import shift
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing, Holt
from function_2 import calculate_technical_indicators


C:\Users\13370\AppData\Local\Temp\ipykernel_83796\435831286.py:5: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [66]:
# Load Data
stock = pd.read_csv('stocks_1.csv')
stock_factor = pd.read_csv('ratios_2.csv')



In [67]:
# Data typr and unit processing
stock['trt1m'] = stock['trt1m'] / 100
stock['datadate'] = pd.to_datetime(stock['datadate'])

# Summary of stocks
stock_summary = stock.groupby('tic').size().reset_index(name='number')
stock_summary = stock_summary[stock_summary['number'] == 168]


In [68]:
# Stocks that experienced a consecutive three-day decline in the last seven days are ruled out
stock_1 = stock[stock['tic'].isin(stock_summary['tic'])].copy()
stock_1 = (stock_1.sort_values(['tic', 'datadate'])
                        .groupby('tic')
                        .apply(lambda x: x.tail(9))
                        .reset_index(drop=True))
stock_1['consecutive_neg'] = (stock_1.groupby('tic')['trt1m']
                                 .transform(lambda x: x.shift(2) < 0) &
                                 stock_1.groupby('tic')['trt1m']
                                 .transform(lambda x: x.shift(1) < 0) &
                                 (stock_1['trt1m'] < 0))
stock_summary_exclude = (stock_1.groupby('tic')
                         .agg(exclude=('consecutive_neg', 'any'))
                         .reset_index())
stock_1_tics = stock_summary_exclude[~stock_summary_exclude['exclude']]['tic']



In [69]:
# Calculate average returns
stock_2 = stock[stock['tic'].isin(stock_1_tics)].copy()
stock_2.dropna(subset=['trt1m'], inplace=True)
stock_2 = stock_2.groupby('tic').head(135)
stock_2['avg'] = stock_2.groupby('tic')['trt1m'].transform('mean')
stock_2 = stock_2.groupby('tic').tail(1).reset_index(drop=True)
stock_2.sort_values(by='avg', ascending=False, inplace=True)

# Select stocks tickers based on average returns
avg_base_select = []
count = np.zeros(100)
gic_all = stock_2['gsector'].unique()

for i in range(277):
    gic_index = stock_2.iloc[i]['gsector']
    count[gic_index] += 1
    if count[gic_index] > 5:
        continue
    avg_base_select.append(stock_2.iloc[i]['tic'])
    if all(count[gic_all] >= 2):
        break

# Select stocks
stock_avg = stock[stock['tic'].isin(avg_base_select)].dropna()
stock_avg['yymm'] = stock_avg['datadate'].dt.to_period('M')

print(stock_avg)

        gvkey iid   datadate   tic                 conm  gsector      cshtrm  \
3672     1913  01 2010-01-31   AVY  AVERY DENNISON CORP       15  28154607.0   
3673     1913  01 2010-02-28   AVY  AVERY DENNISON CORP       15  35229672.0   
3674     1913  01 2010-03-31   AVY  AVERY DENNISON CORP       15  38125453.0   
3675     1913  01 2010-04-30   AVY  AVERY DENNISON CORP       15  33253040.0   
3676     1913  01 2010-05-31   AVY  AVERY DENNISON CORP       15  33787043.0   
...       ...  ..        ...   ...                  ...      ...         ...   
84470  260774  01 2023-08-31  CBRE       CBRE GROUP INC       60  28351879.0   
84471  260774  01 2023-09-30  CBRE       CBRE GROUP INC       60  39349430.0   
84472  260774  01 2023-10-31  CBRE       CBRE GROUP INC       60  36205971.0   
84473  260774  01 2023-11-30  CBRE       CBRE GROUP INC       60  34798008.0   
84474  260774  01 2023-12-31  CBRE       CBRE GROUP INC       60  33954395.0   

       prccm   prchm  prclm     trt1m  

In [70]:
# Clean financial ratios data
stock_factor_1 = stock_factor.drop(columns=[stock_factor.columns[2], stock_factor.columns[3]])
stock_factor_1 = stock_factor_1[stock_factor_1['gvkey'].isin(stock_avg['gvkey'])]
stock_factor_1['yymm'] = pd.to_datetime(stock_factor_1['public_date']).dt.to_period('M')
col_to_fill = stock_factor_1.columns[3:70]

print(stock_factor_1)
print(col_to_fill)

        gvkey  permno public_date   CAPEI     bm     evm  pe_op_dil  pe_exi  \
840     12635   10696  2009-01-31  12.047  0.437  13.152      3.065   3.544   
841     12635   10696  2009-02-28  12.088  0.551   8.119     12.450  15.387   
842     12635   10696  2009-03-31  13.494  0.551   8.119     13.916  17.198   
843     12635   10696  2009-04-30  13.812  0.551   8.119     14.244  17.604   
844     12635   10696  2009-05-31  15.910  0.570  10.578      3.809   4.686   
...       ...     ...         ...     ...    ...     ...        ...     ...   
71989  180711   93002  2022-08-31  36.862  0.093  15.494     24.053  24.733   
71990  180711   93002  2022-09-30  30.171  0.096  14.607     18.766  19.180   
71991  180711   93002  2022-10-31  32.971  0.096  14.607     19.870  20.308   
71992  180711   93002  2022-11-30  38.635  0.096  14.607     23.290  23.803   
71993  180711   93002  2022-12-31  32.267  0.116  12.260     20.663  21.075   

          ps     pcf  ...  sale_nwc  rd_sale  adv_s

In [71]:
# Join trading data and financial ratios data and select proper companies
stock_all = pd.merge(stock_avg, stock_factor_1, on=['yymm', 'gvkey'], how='outer')
stock_all.sort_values(by=['gvkey', 'yymm'], inplace=True)
stock_all[col_to_fill] = stock_all.groupby('gvkey')[col_to_fill].fillna(method='ffill')
stock_all_final = stock_all[(stock_all['yymm'] >= '2010-01') & stock_all['tic'].notna() & ~stock_all['tic'].isin(['BX', 'AVGO'])]

print(stock_all_final)

       gvkey iid   datadate   tic                 conm  gsector      cshtrm  \
0       1913  01 2010-01-31   AVY  AVERY DENNISON CORP     15.0  28154607.0   
1       1913  01 2010-02-28   AVY  AVERY DENNISON CORP     15.0  35229672.0   
2       1913  01 2010-03-31   AVY  AVERY DENNISON CORP     15.0  38125453.0   
3       1913  01 2010-04-30   AVY  AVERY DENNISON CORP     15.0  33253040.0   
4       1913  01 2010-05-31   AVY  AVERY DENNISON CORP     15.0  33787043.0   
...      ...  ..        ...   ...                  ...      ...         ...   
7387  260774  01 2023-08-31  CBRE       CBRE GROUP INC     60.0  28351879.0   
7388  260774  01 2023-09-30  CBRE       CBRE GROUP INC     60.0  39349430.0   
7389  260774  01 2023-10-31  CBRE       CBRE GROUP INC     60.0  36205971.0   
7390  260774  01 2023-11-30  CBRE       CBRE GROUP INC     60.0  34798008.0   
7391  260774  01 2023-12-31  CBRE       CBRE GROUP INC     60.0  33954395.0   

      prccm   prchm  prclm  ...  sale_equity sale_n

In [74]:
# Select features that are meaningful and useful
na_counts = stock_all_final.isna().sum()
stock_final = stock_all_final.loc[:, na_counts == 0]

# Calculate momentum technical indicators
stock_final_1 = stock_final.groupby('gvkey').apply(calculate_technical_indicators).reset_index(drop=True)
stock_use = stock_final_1.dropna().drop(columns=[stock_final_1.columns[11]])
print(stock_use)


       gvkey iid   datadate   tic                 conm  gsector      cshtrm  \
14      1913  01 2011-03-31   AVY  AVERY DENNISON CORP     15.0  23838593.0   
15      1913  01 2011-04-30   AVY  AVERY DENNISON CORP     15.0  14758972.0   
16      1913  01 2011-05-31   AVY  AVERY DENNISON CORP     15.0  14779131.0   
17      1913  01 2011-06-30   AVY  AVERY DENNISON CORP     15.0  25267052.0   
18      1913  01 2011-07-31   AVY  AVERY DENNISON CORP     15.0  30828920.0   
...      ...  ..        ...   ...                  ...      ...         ...   
7051  260774  01 2023-08-31  CBRE       CBRE GROUP INC     60.0  28351879.0   
7052  260774  01 2023-09-30  CBRE       CBRE GROUP INC     60.0  39349430.0   
7053  260774  01 2023-10-31  CBRE       CBRE GROUP INC     60.0  36205971.0   
7054  260774  01 2023-11-30  CBRE       CBRE GROUP INC     60.0  34798008.0   
7055  260774  01 2023-12-31  CBRE       CBRE GROUP INC     60.0  33954395.0   

      prccm   prchm  prclm  ...  high_low_ratio    

C:\Users\13370\AppData\Local\Temp\ipykernel_83796\684014140.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  stock_final_1 = stock_final.groupby('gvkey').apply(calculate_technical_indicators).reset_index(drop=True)


In [75]:
# Standardization

stock_s = stock_use.copy()

scaler = StandardScaler()
stock_s.iloc[:, 6:69] = scaler.fit_transform(stock_s.iloc[:, 6:69])
stock_s.to_csv('stock_standard_2.csv', index=False)


In [76]:
# Normalization

stock_n = stock_use.copy()

normalizer = MinMaxScaler()
stock_n.iloc[:, 6:69] = normalizer.fit_transform(stock_n.iloc[:, 6:69])
stock_n.to_csv('stock_normal_2.csv', index=False)